In [1]:
import pandas as pd
pd.__version__

'2.2.2'

In [33]:
taxi_data_1 = pd.read_parquet('data/yellow_tripdata_2024-01.parquet')
taxi_data_2 = pd.read_parquet('data/yellow_tripdata_2024-02.parquet')

### Homework Question 1:

In [34]:
"There are [{}] columns in the yellow trip data for the month of January.".format(len(taxi_data_1.columns))

'There are [19] columns in the yellow trip data for the month of January.'

In [35]:
'''
Counting the non-NA cells for each column or row
'''
print("Total number of rows {}".format(len(taxi_data_1)))
print()
print('Number of non zero values in each column')
print(taxi_data_1.count())

Total number of rows 2964624

Number of non zero values in each column
VendorID                 2964624
tpep_pickup_datetime     2964624
tpep_dropoff_datetime    2964624
passenger_count          2824462
trip_distance            2964624
RatecodeID               2824462
store_and_fwd_flag       2824462
PULocationID             2964624
DOLocationID             2964624
payment_type             2964624
fare_amount              2964624
extra                    2964624
mta_tax                  2964624
tip_amount               2964624
tolls_amount             2964624
improvement_surcharge    2964624
total_amount             2964624
congestion_surcharge     2824462
Airport_fee              2824462
dtype: int64


### Homework Question 2:

In [37]:
# Convert the columns to datetime if they are not already
taxi_data_1['tpep_pickup_datetime'] = pd.to_datetime(taxi_data_1['tpep_pickup_datetime'])
taxi_data_1['tpep_dropoff_datetime'] = pd.to_datetime(taxi_data_1['tpep_dropoff_datetime'])

# Calculate the duration in minutes
taxi_data_1['duration_in_minutes'] = (taxi_data_1['tpep_dropoff_datetime'] - taxi_data_1['tpep_pickup_datetime']).dt.total_seconds() / 60

# Display the DataFrame with the new duration column
print(taxi_data_1[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'duration_in_minutes']].head())

  tpep_pickup_datetime tpep_dropoff_datetime  duration_in_minutes
0  2024-01-01 00:57:55   2024-01-01 01:17:43            19.800000
1  2024-01-01 00:03:00   2024-01-01 00:09:36             6.600000
2  2024-01-01 00:17:06   2024-01-01 00:35:01            17.916667
3  2024-01-01 00:36:38   2024-01-01 00:44:56             8.300000
4  2024-01-01 00:46:51   2024-01-01 00:52:57             6.100000


In [38]:
taxi_data_1.duration_in_minutes.describe()["std"]

34.851053592212814

### Homework Question 3:

In [45]:
taxi_data_1.duration_in_minutes.describe()

count    2.964624e+06
mean     1.561295e+01
std      3.485105e+01
min     -1.356667e+01
25%      7.150000e+00
50%      1.163333e+01
75%      1.868333e+01
max      9.455400e+03
Name: duration_in_minutes, dtype: float64

In [51]:
# Remove outliers (keep records where the duration was between 1 and 60 minutes inclusive)
taxi_filtered = taxi_data_1[(taxi_data_1['duration_in_minutes'] >= 1) & (taxi_data_1['duration_in_minutes'] <= 60)]

In [58]:
print("The number of records left after filtering {}%".format(round(len(taxi_filtered) / len(taxi_data_1) * 100), 4))

The number of records left after filtering 98%


### Homework Question 4: